# A dataset about dogs.

Data from [a FOIL request to New York City](https://www.muckrock.com/foi/new-york-city-17/pet-licensing-data-for-new-york-city-23826/)

## Do your importing and your setup

In [1]:
import pandas as pd
%matplotlib inline

## Read in the file `NYC_Dog_Licenses_Current_as_of_4-28-2016.xlsx` and look at the first five rows

In [2]:
df = pd.read_excel("NYC_Dog_Licenses_Current_as_of_4-28-2016.xlsx", na_values = ["???"]) 
df = df.head(30000) 

FileNotFoundError: [Errno 2] No such file or directory: 'NYC_Dog_Licenses_Current_as_of_4-28-2016.xlsx'

## How many rows do you have in the data? What are the column types?

If there are more than 30,000 rows in your dataset, go back and only read in the first 30,000.

In [ ]:
df.columns

In [ ]:
df.shape

## Describe the dataset in words. What is each row? List two column titles along with what each of those columns means.

For example: “Each row is an animal in the zoo. `is_reptile` is whether the animal is a reptile or not”

In [ ]:
#Each row is a different pet dog. 
#The column Vaccinated says if the dog has been vaccinated or not 
#The column Owner Zip Code is where the dog's owner lives in NYC 

# Your thoughts

Think of four questions you could ask this dataset. **Don't ask them**, just write them down in the cell below.

In [ ]:
#How many different primary breeds of dogs live in XXXX zip code/Brooklyn? 
#What percentage of dogs are trained and vaccinated? 
#What zip codes do the most dachsunds live? 
#What is the most common dog name in NYC? 

# Looking at some dogs

## What are the most popular (primary) breeds of dogs? Graph the top 10.

In [ ]:
df['Primary Breed'].value_counts().head(10)  

## "Unknown" is a terrible breed! Graph the top 10 breeds that are NOT Unknown

In [ ]:
df[df['Primary Breed'] != 'Unknown']['Primary Breed'].value_counts().head(10)

## What are the most popular dog names?

In [ ]:
df.columns

In [ ]:
#Still have unknown (uncapitalized)...is still included. Is there a way to ignore 2 things at a time? 
df[df['Animal Name'] != 'UNKNOWN']['Animal Name'].value_counts().head() 

## Do any dogs have your name? How many dogs are named "Max," and how many are named "Maxwell"?

In [ ]:
#(df['Animal Name'] == 'Maija').value_counts() 
df['Animal Name'].str.contains("Maija", na = False).value_counts()  
#Since we know there are 30000 in the excel file uploaded and it says we have 30000 False, 
#there are no dogs have the name Maija 

In [ ]:
(df['Animal Name'] == 'Max').value_counts()  

In [ ]:
(df['Animal Name'] == 'Maxwell').value_counts()   

## What percentage of dogs are guard dogs?

Check out the documentation for [value counts](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html).

In [ ]:
df.columns
df['Guard or Trained'].value_counts(normalize = True, dropna=False)   

## What are the actual numbers?

In [ ]:
df['Guard or Trained'].value_counts(dropna=False) 

## Wait... if you add that up, is it the same as your number of rows? Where are the other dogs???? How can we find them??????

Use your `.head()` to think about it, then you'll do some magic with `.value_counts()`

In [ ]:
#I think this is answered above

## Fill in all of those empty "Guard or Trained" columns with "No"

Then check your result with another `.value_counts()`

In [ ]:
df['Guard or Trained'].fillna("No", inplace = True) 

In [ ]:
df['Guard or Trained'].value_counts() 

## What are the top dog breeds for guard dogs? 

In [ ]:
df.columns


In [ ]:
#I think this is wrong 
guard_dogs = df[df['Guard or Trained'] == 'Yes']  
#guard_dogs
guard_dogs.groupby('Guard or Trained')['Primary Breed'].value_counts().head() 

## Create a new column called "year" that is the dog's year of birth

The `Animal Birth` column is a datetime, so you can get the year out of it with the code `df['Animal Birth'].apply(lambda birth: birth.year)`.

In [ ]:
df['year'] = df['Animal Birth'].apply(lambda birth: birth.year)  
df
#df['Animal Birth'] = df[''.str.replace("%","")

## Calculate a new column called “age” that shows approximately how old the dog is. How old are dogs on average?

In [ ]:
df['age'] = (2017 - df['year']) 
df

In [ ]:
df['age'].mean() 

# Joining data together

## Which neighborhood does each dog live in?

You also have a (terrible) list of NYC neighborhoods in `zipcodes-neighborhoods.csv`. Join these two datasets together, so we know what neighborhood each dog lives in. **Be sure to not read it in as `df`, or else you'll overwrite your dogs dataframe.**

In [ ]:
df2 = pd.read_csv("zipcodes-neighborhoods.csv", na_values = ["???"])    
df2['borough'].value_counts() 

In [ ]:
combined_df = df.merge(df2, left_on="Owner Zip Code", right_on="zip")
combined_df 

## What is the most popular dog name in all parts of the Bronx? How about Brooklyn? The Upper East Side?

In [ ]:
#Most popular name in the Bronx 
bronx_names = combined_df[combined_df['borough'].str.contains("Bronx", na=False)]
bronx_names.groupby('borough')['Animal Name'].value_counts().groupby(level=0).head() 

In [ ]:
#Most popular name in Brooklyn 
brooklyn_names = combined_df[combined_df['borough'].str.contains("Brooklyn", na=False)]
brooklyn_names.groupby('borough')['Animal Name'].value_counts().groupby(level=0).head()   

In [ ]:
#Most popular name on the Upper East Side 
ues_names = combined_df[combined_df['neighborhood'].str.contains('Upper East Side', na=False)] 
ues_names.groupby('neighborhood')['Animal Name'].value_counts().groupby(level=0).head() 

## What is the most common dog breed in each of the neighborhoods of NYC?

In [ ]:
combined_df.groupby('borough')['Animal Name'].value_counts().groupby(level=0).head() 

## What breed of dogs are the least likely to be spayed? Male or female?

In [ ]:
combined_df.columns

In [ ]:
combined_df.groupby('Animal Gender')['Spayed or Neut'].value_counts().groupby(level=0).head() 

In [ ]:
#Number of dogs of each breed 
combined_df['Primary Breed'].value_counts().head(25) 


In [ ]:
#How many of each breed are not fixed 
not_spayed = combined_df[combined_df['Spayed or Neut'].str.contains('No', na = False)] 
not_spayed.groupby('Primary Breed')['Spayed or Neut'].value_counts().sort_values(ascending = False).head(25)   

In [ ]:
#divide the second number by the first number and sort it 

## Make a new column called monochrome that is True for any animal that only has black, white or grey as one of its colors. How many animals are monochrome?

In [ ]:
colors = ['WHITE', 'white', 'BLACK', 'black', 'gray', 'GREY', 'GRAY'] 
#combined_df['monochrome'] = 
combined_df['monochrome'] = combined_df['Animal Dominant Color'].isin(colors) & (combined_df['Animal Secondary Color'].isnull()) & (df['Animal Third Color'].isnull())
#combined_df['monochrome'].value_counts() 
combined_df['monochrome'].value_counts() 

 

## How many dogs are in each borough? Plot it in a graph.

In [ ]:
combined_df.columns
combined_df['borough'].value_counts().plot(kind = 'barh')  


## Which borough has the highest number of dogs per-capita?

You’ll need to merge in `population_boro.csv`

In [ ]:
another_df = pd.read_csv("boro_population.csv", na_values = ["???"])    
another_df.columns 
#combined_df.columns

In [ ]:
three_df = another_df.merge(combined_df, left_on="borough", right_on="borough") 
three_df.groupby('population')['borough'].value_counts()  

#population of each borough 

In [ ]:
#dogs per borough 
three_df.columns 
three_df['borough'].value_counts() 


In [ ]:
#I see how you get the population per borough and how you get the dogs per borough. But how do you divide them (without just doing it by hand!)

#manhattan_dogs = three_df[three_df['borough'].str.contains("Manhattan"), na=False)] 
#manhattan_pop = three_df[three_df[b]]
#manhattan_dogs / 

## Make a bar graph of the top 5 breeds in each borough.

How do you groupby and then only take the top X number? You **really** should ask me, because it's kind of crazy.

## What percentage of dogs are not guard dogs?